In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from typing import Tuple

In [ ]:
def rref(A):
    U = np.copy(A)
    (m,n)=A.shape
    j = 0
    p =[]
    for i in range(0,m): 
        ech=1
        while (ech == 1) & (j < n):
          indm=np.argmax(abs(U[i:m,j]))
          indm=indm+i
          if  (abs(U[indm,j]) == 0): # perform the permutation
             U[ [i, indm],:]=U[[indm,i],:] 
          if ( abs(U[i,j]) > 0):
             p.append(j)
             M=U[i+1:m,j]/U[i,j]
             U[i+1:m,j+1:n]=U[i+1:m,j+1:n]-np.outer(M,U[i,j+1:n])
             U[i+1:m,j]=0           
             U[i,j:n] = U[i,j:n]/U[i,j]   # the pivotal element should be 1        
             if i>0:
               M= U[0:i,j]/U[i,j]
               U[0:(i),j:n]=U[0:i,j:n]-np.outer(M,U[i,j:n])     
             j=j+1
             ech=0
          else:
            j=j+1
            ech=1
            
    return(U,p)    

<details>
<summary>RREF EXPLANATION</summary>

## 🔍 Comprehensive Analysis: Reduced Row Echelon Form (RREF) Algorithm

### 📌 Algorithm Purpose & Mathematical Foundation
This function computes the **Reduced Row Echelon Form (RREF)** of a matrix using **Gaussian elimination with partial pivoting**, extended to achieve *reduced* form (zeros above *and* below pivots). RREF is fundamental in:
- Solving linear systems $A\mathbf{x} = \mathbf{b}$
- Computing matrix rank and null space
- Determining linear independence of vectors
- Finding basis transformations

**Key RREF properties**:
1. All nonzero rows are above zero rows
2. Leading entry (pivot) of each nonzero row is 1
3. Each pivot is strictly to the right of the pivot above it
4. All entries in a pivot column are zero except the pivot itself

---

### 🧩 Line-by-Line Breakdown

#### **Initialization Phase**
```python
U = np.copy(A)
(m,n) = A.shape
j = 0
p = []
```
- `U`: Working copy of input matrix (preserves original `A`)
- `(m,n)`: Matrix dimensions ($m$ rows, $n$ columns)
- `j`: Current column index being processed (pivot search column)
- `p`: List storing **pivot column indices** (critical for rank/null space computation)

#### **Main Loop: Row Processing**
```python
for i in range(0, m):
```
- Processes rows sequentially from top to bottom (`i` = current pivot row)
- At most `min(m,n)` pivots possible

#### **Pivot Search & Column Advancement**
```python
ech = 1
while (ech == 1) & (j < n):
```
- `ech` ("*échelon*") flag controls pivot search loop
- Continues searching rightward (`j++`) until:
  - A valid pivot is found in column `j`, OR
  - All columns exhausted (`j >= n`)

#### **Partial Pivoting (Numerical Stability)**
```python
indm = np.argmax(abs(U[i:m, j]))
indm = indm + i
```
- Finds row index `indm` with **maximum absolute value** in column `j` from row `i` downward
- Critical for **numerical stability**: minimizes round-off error by avoiding division by small numbers
- Example: In column `[0.001, 1000]`, selects `1000` as pivot instead of `0.001`

#### **Zero Column Handling (Edge Case)**
```python
if (abs(U[indm, j]) == 0):
    U[[i, indm], :] = U[[indm, i], :]
```
- ⚠️ **Bug alert**: This swap is *redundant* when `abs(U[indm,j])==0` (all entries zero). Should skip column instead.
- Correct behavior would be `j += 1; continue` (handled later via `else` branch)

#### **Pivot Normalization & Elimination**
```python
if (abs(U[i, j]) > 0):
    p.append(j)  # Record pivot column index
```
- Confirms valid pivot exists at `(i,j)`
- Records pivot column for later rank analysis

```python
M = U[i+1:m, j] / U[i, j]
U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
U[i+1:m, j] = 0
```
- **Forward elimination**: Zeroes entries *below* pivot
- `M`: Multipliers for row operations (equivalent to $m_{k} = a_{kj}/a_{ij}$)
- `np.outer(M, U[i, j+1:n])`: Efficiently computes $m_k \times \text{pivot row}$ for all rows simultaneously
- Sets entire column below pivot to zero explicitly (avoids floating-point residue)

```python
U[i, j:n] = U[i, j:n] / U[i, j]  # Normalize pivot row
```
- **Pivot normalization**: Scales row so pivot element becomes exactly 1
- Transforms $a_{ij} \rightarrow 1$ (required for RREF)

```python
if i > 0:
    M = U[0:i, j] / U[i, j]
    U[0:i, j:n] = U[0:i, j:n] - np.outer(M, U[i, j:n])
```
- **Backward elimination**: Zeroes entries *above* pivot (distinguishes RREF from REF)
- Critical for reduced form: ensures pivot columns form identity submatrix
- Only needed for rows above current pivot (`i > 0`)

```python
j = j + 1
ech = 0
```
- Advances to next column after successful pivot processing
- Exits pivot search loop (`ech=0`)

#### **Zero Column Skip**
```python
else:
    j = j + 1
    ech = 1
```
- Handles columns with all zeros below current row
- Advances column index without modifying matrix

#### **Return Values**
```python
return(U, p)
```
- `U`: Matrix in reduced row echelon form
- `p`: List of pivot column indices (0-based)
  - Length of `p` = **rank** of matrix
  - Pivot columns indicate linearly independent columns of original matrix

---

### ⚠️ Critical Stability & Edge Case Considerations

| Issue | Description | Impact |
|-------|-------------|--------|
| **Zero pivot swap bug** | Swaps rows even when entire column is zero | Wasted operation; no functional error but inefficient |
| **No tolerance threshold** | Uses `abs(x) == 0` instead of `abs(x) < tol` | Fails for near-zero pivots due to floating-point errors |
| **No rank-deficient handling** | Doesn't explicitly handle $m > n$ or rank-deficient cases | Works correctly but may leave trailing zero rows |
| **Partial pivoting only** | No scaled/complete pivoting | Sufficient for most cases; may fail for ill-conditioned matrices |

**Recommended fix for numerical robustness**:
```python
tol = 1e-12  # Add near top of function
if abs(U[indm, j]) < tol:  # Replace == 0 checks
```

---

### 📊 Computational Complexity
- **Time**: $O(mn \cdot \min(m,n))$ 
  - Outer loop: $O(\min(m,n))$ pivots
  - Each pivot: $O(mn)$ operations for row updates
- **Space**: $O(mn)$ (copy of input matrix)
- **Optimality**: Matches theoretical lower bound for dense matrix RREF

---

### 💡 Practical Usage Examples

#### Example 1: Solve linear system
```python
import numpy as np
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]], dtype=float)
b = np.array([[8], [-11], [-3]], dtype=float)
Ab = np.hstack([A, b])  # Augmented matrix

R, pivots = rref(Ab)
# Solution in last column of R (if consistent system)
print("Solution:", R[:3, 3])  # [2. 3. -1.]
```

#### Example 2: Compute matrix rank
```python
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]], dtype=float)
_, pivots = rref(A)
print("Rank:", len(pivots))  # 2 (singular matrix)
```

#### Example 3: Null space basis
```python
# For matrix A with pivots p and free variables f:
# Null space vectors constructed from RREF relationships
# (Requires additional post-processing of R)
```

---

### 🎓 Pedagogical Notes for Students

1. **Why partial pivoting?**  
   Without pivoting, elimination on $\begin{bmatrix} \epsilon & 1 \\ 1 & 1 \end{bmatrix}$ ($\epsilon \approx 10^{-16}$) causes catastrophic cancellation. Pivoting swaps rows to use 1 as pivot.

2. **RREF vs REF distinction**  
   - REF (Row Echelon Form): Zeros *only below* pivots  
   - RREF (Reduced REF): Zeros *above and below* pivots + pivots normalized to 1  
   → RREF provides *unique* representation for any matrix

3. **Pivot columns reveal structure**  
   Pivot indices `p` directly identify:
   - Rank = `len(p)`
   - Basis columns of original matrix = columns at indices `p`
   - Free variables = columns not in `p`

4. **Floating-point reality**  
   Always use tolerance checks (`abs(x) < 1e-12`) instead of exact zero comparisons in numerical work. The provided code's `== 0` check is theoretically correct but numerically fragile.

This implementation efficiently balances pedagogical clarity with practical numerical linear algebra techniques—making it an excellent reference for understanding the mechanics of Gaussian elimination extended to reduced form.
</details>

In [ ]:
def rref_verbose(matrix: np.ndarray, tolerance: float = 1e-12, max_iterations: int = 1000) -> Tuple[np.ndarray, list]:
    """
    Compute Reduced Row Echelon Form (RREF) with pedagogical logging and robust error handling.
    
    Algorithm: Gaussian elimination with partial pivoting followed by backward elimination
    to produce leading 1s with zeros above and below each pivot.
    
    Parameters
    ----------
    matrix : np.ndarray
        Input 2D matrix (m rows × n columns)
    tolerance : float
        Numerical tolerance for treating values as zero (handles floating-point errors)
    max_iterations : int
        Safety limit to prevent infinite loops (should never trigger for valid input)
    
    Returns
    -------
    rref_matrix : np.ndarray
        Matrix transformed to reduced row echelon form
    pivot_columns : list[int]
        Column indices containing leading 1s (pivots)
    
    Raises
    ------
    ValueError : For invalid inputs (non-2D, empty, non-numeric)
    RuntimeError : If algorithm exceeds iteration safety limit
    """
    # ===== ERROR HANDLING =====
    if not isinstance(matrix, np.ndarray):
        raise ValueError(f"Input must be a NumPy ndarray, got {type(matrix)}")
    
    if matrix.ndim != 2:
        raise ValueError(f"Input must be 2-dimensional, got {matrix.ndim} dimensions")
    
    if matrix.size == 0:
        raise ValueError("Input matrix cannot be empty")
    
    if not np.issubdtype(matrix.dtype, np.number):
        raise ValueError(f"Matrix must contain numeric values, got dtype {matrix.dtype}")
    
    # Create working copy to avoid modifying original
    rref_matrix = np.array(matrix, dtype=float)
    num_rows, num_cols = rref_matrix.shape
    
    print(f"✓ Starting RREF computation on {num_rows}×{num_cols} matrix")
    print(f"  Initial matrix:\n{rref_matrix}\n")
    
    current_row = 0
    pivot_columns = []
    iteration_count = 0
    
    # ===== FORWARD ELIMINATION + BACKWARD ELIMINATION =====
    for current_col in range(num_cols):
        iteration_count += 1
        if iteration_count > max_iterations:
            raise RuntimeError(f"Exceeded max iterations ({max_iterations}). Possible algorithmic error.")
        
        # STEP 1: Find pivot candidate using partial pivoting (max absolute value)
        submatrix = rref_matrix[current_row:, current_col]
        pivot_row_offset = np.argmax(np.abs(submatrix))
        pivot_row = current_row + pivot_row_offset
        pivot_value = rref_matrix[pivot_row, current_col]
        
        # STEP 2: Skip column if no valid pivot exists (all values near zero)
        if np.abs(pivot_value) < tolerance:
            print(f"  Column {current_col}: No pivot found (max |value| = {np.max(np.abs(submatrix)):.2e} < {tolerance:.0e})")
            continue
        
        # STEP 3: Swap current row with pivot row if needed
        if pivot_row != current_row:
            print(f"  ↕ Swapping rows {current_row} ↔ {pivot_row} to bring pivot to diagonal")
            rref_matrix[[current_row, pivot_row], :] = rref_matrix[[pivot_row, current_row], :]
        
        # STEP 4: Normalize pivot row to make pivot element = 1
        pivot_value = rref_matrix[current_row, current_col]  # Updated after swap
        rref_matrix[current_row, :] /= pivot_value
        print(f"  → Normalized row {current_row}: divided by pivot {pivot_value:.4g}")
        
        # STEP 5: Eliminate all other entries in pivot column (above AND below)
        for other_row in range(num_rows):
            if other_row != current_row:
                multiplier = rref_matrix[other_row, current_col]
                if np.abs(multiplier) > tolerance:  # Skip near-zero multipliers
                    rref_matrix[other_row, :] -= multiplier * rref_matrix[current_row, :]
                    print(f"    Eliminated row {other_row} using row {current_row} (multiplier = {multiplier:.4g})")
        
        pivot_columns.append(current_col)
        print(f"  ✓ Pivot established at (row={current_row}, col={current_col})\n")
        current_row += 1
        
        # Termination condition: all rows processed
        if current_row >= num_rows:
            print(f"  → All {num_rows} rows processed. Terminating early.")
            break
    
    # ===== POST-PROCESSING: Clean near-zero values =====
    rref_matrix[np.abs(rref_matrix) < tolerance] = 0.0
    
    print(f"\n✓ RREF computation complete")
    print(f"  Pivot columns: {pivot_columns}")
    print(f"  Matrix rank: {len(pivot_columns)}")
    print(f"  Final RREF matrix:\n{rref_matrix}")
    
    return rref_matrix, pivot_columns

In [ ]:
'''Solve:
    2x + y - z = 8
    -3x - y + 2z = -11
    -2x + y + 2z = -3
    Known solution: x=2, y=3, z=-1
'''

# Augmented matrix [A|b]
A1 = np.array([
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3]
])

print("="*60)
print("EXAMPLE 1: Textbook 3×3 Linear System")
print("="*60)
rref1, pivots1 = rref_verbose(A1)

# Extract solution from last column
solution = rref1[:, -1][:3]  # First 3 rows, last column
expected = np.array([2, 3, -1])
error = np.linalg.norm(solution - expected)

print(f"\nComputed solution: x={solution[0]:.6f}, y={solution[1]:.6f}, z={solution[2]:.6f}")
print(f"Expected solution: x={expected[0]}, y={expected[1]}, z={expected[2]}")
print(f"Absolute error: {error:.2e}")
print(f"Verification: {'✓ PASS' if error < 1e-10 else '✗ FAIL'}")

In [ ]:
'''Node equations:
  3I₁ - I₂ - I₃ = 5   (Node A)
  -I₁ + 4I₂ - 2I₃ = 0  (Node B)
  -I₁ - 2I₂ + 5I₃ = 0  (Node C)
'''

print("\n" + "="*60)
print("EXAMPLE 2: Electrical Circuit Analysis (KCL)")
print("="*60)

A2 = np.array([
    [3, -1, -1, 5],
    [-1, 4, -2, 0],
    [-1, -2, 5, 0]
])

rref2, pivots2 = rref_verbose(A2)

# Extract currents
currents = rref2[:, -1][:3]
print(f"\nComputed branch currents:")
print(f"  I₁ = {currents[0]:.4f} A")
print(f"  I₂ = {currents[1]:.4f} A")
print(f"  I₃ = {currents[2]:.4f} A")

# Verify by plugging back into original equations
original_system = np.array([[3, -1, -1], [-1, 4, -2], [-1, -2, 5]])
residuals = original_system @ currents - np.array([5, 0, 0])
print(f"Residuals (should be near zero): {residuals}")
print(f"Max residual: {np.max(np.abs(residuals)):.2e}")

In [ ]:
'''Test detection of inconsistency (0 = non-zero in augmented column):'''

print("\n" + "="*60)
print("EXAMPLE 3: Edge Case - Inconsistent System")
print("="*60)

A3 = np.array([
    [1, 2, 3, 4],
    [2, 4, 6, 8],   # Row 2 = 2 × Row 1 → dependent
    [3, 6, 9, 10]   # Row 3 = 3 × Row 1 but 3×4 ≠ 10 → inconsistent!
])

rref3, pivots3 = rref_verbose(A3)

# Detect inconsistency: row of form [0 0 ... 0 | c] where c ≠ 0
for i, row in enumerate(rref3):
    if np.all(np.abs(row[:-1]) < 1e-12) and np.abs(row[-1]) > 1e-12:
        print(f"\n⚠️ INCONSISTENCY DETECTED in row {i}:")
        print(f"   0·x₁ + 0·x₂ + ... + 0·xₙ = {row[-1]:.4f}  →  NO SOLUTION EXISTS")
        break
else:
    print("\n✓ System is consistent")